# Config

In [1]:
# Import dependencies
import pandas as pd
from datetime import datetime
from sleeperpy import User, Leagues
from ffpackage.viz import compareFranchises
import plotly
import plotly.express as px

In [2]:
# Settings
date_string = datetime.today().strftime('%Y%m%d')
weekStart = 1
weekEnd = 17
def_starting_pts = 18
pr_bonus = 1
kr_bonus = 3
username = 'dirtywizard'

# Extract

In [3]:
# Read database
preds = pd.read_csv('db/records_preds.csv')
preds = preds.loc[preds['date_prediction']==preds['date_prediction'].max()]
preds = preds.loc[(preds['week']>=weekStart)&(preds['week']<=weekEnd)]
mults = pd.read_csv('db/pts_multipliers.csv')
rostLims = pd.read_csv('db/roster_limits.csv')

In [4]:
def processRosters(league_id):
    # Get rosters for all league members
    rosters = pd.DataFrame(Leagues.get_rosters(league_id))
    # Explode the player columns to find how they are placed on the roster
    result = pd.DataFrame(columns=['owner_id', 'player_id'])
    for col in ['taxi', 'starters', 'players']:
        df = rosters[['owner_id', col]].explode(col)
        df = df.rename(columns={col:'player_id'})
        df[col] = True
        result = result.merge(df, how='outer', on=['owner_id', 'player_id'])
    return result

In [5]:
def procesOwners(league_id):
    owners = pd.DataFrame(Leagues.get_users(league_id))
    list = []
    for i in range(len(owners)):
        if 'team_name' in owners['metadata'][i]:
            list.append(owners['metadata'][i]['team_name'])
        else:
            list.append(owners['display_name'][i])
    owners['franchise_name'] = list
    owners = owners[['user_id', 'franchise_name', 'display_name']]
    return owners

In [51]:
# Get User's Sleeper ID
user_id = User.get_user(username)['user_id']
# Get the User's Sleeper league ID
league_id = Leagues.get_all_leagues(user_id, 'nfl', 2023)[0]['league_id']
# Get all players on rosters
rosters = processRosters(league_id)
# Get info on all owners
owners = procesOwners(league_id)
# Get league info
user_league = Leagues.get_league(league_id)

# Get roster positions 
rosterList = user_league['roster_positions']
# Sort rosterList
sort_order = ['QB', 'RB', 'WR', 'TE', 'K', 'DEF', 'FLEX', 'SUPER_FLEX', 'BN']
rosterList.sort(key=sort_order.index)

# Get number of franchises
number_of_franchises = user_league['total_rosters']

# Transform

In [7]:
# Customize predictions for league
# Calculate projected points according to that league's scoring settings
# Multiply by points multiplier
for col in mults.columns[1:]:
    preds[col] = preds[col] * mults.loc[mults['username']==username, col][0]
# Add up point columns
preds['pts_proj'] = preds[mults.columns[1:]].sum(axis=1)
# Add extras for defense starting values
preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] = preds.loc[preds['Yds Allowed'].notna(), 'pts_proj'] + def_starting_pts
# Add extras for Punt/kick returners
preds.loc[preds['PR']==True, 'pts_proj'] = preds.loc[preds['PR']==True, 'pts_proj'] + (pr_bonus)
preds.loc[preds['KR']==True, 'pts_proj'] = preds.loc[preds['KR']==True, 'pts_proj'] + (kr_bonus)

# Merge in point projections with franchise data
full = preds \
    .merge(rosters, how='left', on='player_id') \
    .merge(owners, how='left', left_on='owner_id', right_on='user_id')

In [8]:
# def getRelProjs(df):
#     # Calculate relative points for each position in each week
#     df['pts_proj'] = df['pts_proj'].fillna(0)
#     df = df.sort_values(by='pts_proj', ascending=False, ignore_index=True)
#     # Loop through weeks and positions
#     for week in df['week'].unique():
#         for pos in rostLims.columns[1:]:
#             # Find floor
#             cond1 = df['position']==pos
#             cond2 = df['week']==week
#             floor = df.loc[cond1&cond2, 'pts_proj'].reset_index(drop=True)[rostLims.loc[rostLims['username']==username, pos][0]]
#             # Find each player's points over floor
#             df.loc[cond1&cond2, 'rel_proj'] = df.loc[cond1&cond2, 'pts_proj'] - floor
#     return df

In [9]:
def getRelProjs(df):
    # Calculate relative points for each position in each week
    df['pts_proj'] = df['pts_proj'].fillna(0)
    df = df.sort_values(by='pts_proj', ascending=False, ignore_index=True)
    # Loop through weeks and positions
    for week in df['week'].unique():
        for pos in df['position'].unique():
            # Find floor
            cond_pos = df['position']==pos
            cond_week = df['week']==week
            floor = df.loc[cond_pos&cond_week, 'pts_proj'].min()
            # Find each player's points over floor
            df.loc[cond_pos&cond_week, 'rel_proj'] = df.loc[cond_pos&cond_week, 'pts_proj'] - floor
    return df

In [10]:
def getSeasonTotals(df):
    # Sum up pts_proj and rel_proj
    totals_proj = df.groupby('player_id')['pts_proj'].sum().reset_index()
    totals_rel = df.groupby('player_id')['rel_proj'].sum().reset_index()
    # Merge in totals with player metadata
    unis = df.drop_duplicates(subset='player_id', ignore_index=True).drop(columns=['pts_proj', 'rel_proj'])
    df = unis.merge(totals_proj, how='left', on='player_id').merge(totals_rel, how='left', on='player_id')
    return df

In [11]:
def tidyUp(df):
    # Tidy
    df = df[[
        'week', 'player_id', 'pts_proj', 'rel_proj', 'franchise_name', 'full_name', 'age', 'years_exp', 'weight',
        'height', 'team', 'position', 'taxi', 'starters', 'players', 'fantasy_positions', 'depth_chart_order',
        'depth_chart_position', 'status', 'injury_status', 'injury_notes',
        'injury_start_date', 'practice_participation', 'practice_description','active'
    ]]
    return df

In [53]:
def findFloors(df, rosterList, number_of_franchises):
    # Find floors for each position to calculate relative projections
    # Roster Dictionary
    rosterDict = {
        'QB':['QB'], 
        'RB':['RB'],
        'WR':['WR'],
        'TE':['TE'],
        'K':['PK'],
        'DEF':['DEF'],
        'FLEX':['RB', 'WR', 'TE'],
        'SUPER_FLEX':['QB', 'RB', 'WR', 'TE'],
    }
    # Sort
    sorted = df.sort_values(by='pts_proj', ignore_index=True, ascending=False)
    # Start selecting the most valuable players in terms of total points for each position
    result = []
    for pos in rosterList:
        # Ignore bench spots
        if pos != "BN":
            # Filter for each position and make sure the player is not already taken
            cond_pos = sorted['position'].isin(rosterDict[pos])
            cond_nottaken = ~sorted['player_id'].isin(result)
            player_filtered = sorted.loc[cond_pos & cond_nottaken]
            # Find highest scoring player that fits
            if len(player_filtered)>0:
                players = player_filtered.head(number_of_franchises)['player_id']
                result.append(players)
    floors = df.loc[df['player_id'].isin(result)]
    floor_qb = floors.loc[floors['position']=='QB']['pts_proj'].min()
    floor_rb = floors.loc[floors['position']=='RB']['pts_proj'].min()
    floor_wr = floors.loc[floors['position']=='WR']['pts_proj'].min()
    floor_te = floors.loc[floors['position']=='TE']['pts_proj'].min()
    floor_pk = floors.loc[floors['position']=='PK']['pts_proj'].min()
    floor_def = floors.loc[floors['position']=='DEF']['pts_proj'].min()
    return floor_qb, floor_rb, floor_wr, floor_te, floor_pk, floor_def

In [12]:
def rosterOptimizer(df):
    # choose a roster for each franchise based on their league's number of available slots
    optimized = pd.DataFrame()
    # Roster Dictionary
    rosterDict = {
        'QB':['QB'], 
        'RB':['RB'],
        'WR':['WR'],
        'TE':['TE'],
        'K':['PK'],
        'DEF':['DEF'],
        'FLEX':['RB', 'WR', 'TE'],
        'SUPER_FLEX':['QB', 'RB', 'WR', 'TE'],
    }
    # Sort
    cond_taxi = df['taxi'].isna()
    sorted = df.loc[cond_taxi].sort_values(by='pts_proj', ignore_index=True, ascending=False)
    # Loop through weeks and franchises
    for this_user in df['user_id'].unique():
        for week_to_optimize in df['week'].unique():
            # Filter predictions for only that week and on a roster
            cond_franch = sorted['user_id']==this_user
            cond_week = sorted['week']==week_to_optimize
            week_filtered = sorted.loc[cond_franch & cond_week]
            # Start selecting the most valuable players in terms of total points for each position
            result = []
            for pos in rosterList:
                # Ignore bench spots
                if pos != "BN":
                    # Filter for each position and make sure the player is not already taken
                    cond_pos = week_filtered['position'].isin(rosterDict[pos])
                    cond_nottaken = ~week_filtered['player_id'].isin(result)
                    player_filtered = week_filtered.loc[cond_pos & cond_nottaken]
                    # Find highest scoring player that fits
                    if len(player_filtered)>0:
                        player = player_filtered.reset_index(drop=True)['player_id'][0]
                        result.append(player)
            # Append results
            next = week_filtered.loc[week_filtered['player_id'].isin(result)]
            optimized = pd.concat([optimized, next], axis=0, ignore_index=True)
    return optimized

# Waiver Report

In [13]:
# Waiver report
waivers = full.copy()
# Calculate relative points for each position in each week
waivers = getRelProjs(waivers)
# Sum points across the season
waivers = getSeasonTotals(waivers)
# Filter for only players on user's roster or unclaimed and who would improve user's points
minpts = waivers.loc[(waivers['display_name']==username)&(waivers['taxi'].isna())]['pts_proj'].min()
cond_user = waivers['display_name']==username
cond_fa = waivers['display_name'].isna()
cond_minpts = waivers['pts_proj']>=minpts
waivers = waivers.loc[(cond_user|cond_fa)&cond_minpts].reset_index(drop=True)
# Tidy
waivers = tidyUp(waivers)

# Visualize Total Points

In [14]:
# Summative report
player_sums = full.copy()
# Filter
player_sums.loc[(player_sums['starters'].notna())&(player_sums['franchise_name'].notna())]
# Calculate relative points for each position in each week
player_sums = getRelProjs(player_sums)
# Sum points across the season
player_sums = getSeasonTotals(player_sums)

# Visualize
fig = compareFranchises(player_sums, how='relative')
fig.show()

In [15]:
# Visualize
fig = compareFranchises(player_sums, how='absolute')
fig.show()

# Roster Optimizer

In [16]:
#from ffpackage.analysis import starterSelector
# Choose a roster for each franchise based on their league's number of available slots
optimized = rosterOptimizer(full)
# Get relative points
optimized = getRelProjs(optimized)
# Sum points across the season
optimized = getSeasonTotals(optimized)
# Visualize
fig = compareFranchises(optimized, how='absolute')
fig.show()

### Left Off Here:
##### Build trade report
##### Tailor scoring for user's unique league
##### Need to figure out how to convert rosterList (number of position slots available) into relative values. Ideas:
 - Standard deviation / Zscore
 - Measure down from the top rather than up from the bottom, 1/(xMax-x)
 - hard-coded numbers of slots
 - percentage of overall points

In [ ]:
floor_qb, floor_rb, floor_wr, floor_te, floor_pk, floor_def = findFloors(full)

In [24]:
rosterList

['QB',
 'RB',
 'RB',
 'WR',
 'WR',
 'WR',
 'TE',
 'TE',
 'FLEX',
 'FLEX',
 'SUPER_FLEX',
 'K',
 'K',
 'DEF',
 'DEF',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN',
 'BN']

In [34]:
countList = {'QB':2*12,
    'RB':5*12,
    'WR':6*12,
    'TE':5*12,
}

countListPartial = {
'QB':int(round(1.25*12, 0)),
'RB':int(round(2.9*12, 0)),
'WR':int(round(3.9*12, 0)),
'TE':int(round(2.9*12, 0)),}

In [45]:
full['pts_proj'].sum()

47762.319500000005

In [41]:
test = full.copy()
test = getRelProjs(test)
test = getSeasonTotals(test)
test = test.sort_values(by='pts_proj', ignore_index=True, ascending=False)
cond_pospts = test['pts_proj']>0

for pos in ['QB', 'RB', 'WR', 'TE']:
    print(f'{pos} Mean: ' + str(test.loc[(test['position']==pos) & cond_pospts]['pts_proj'].mean().round(2)))
    print(f'{pos} Median: ' + str(test.loc[(test['position']==pos) & cond_pospts]['pts_proj'].median().round(2)))
    print(f'{pos} Spot #{countListPartial[pos]}: ' + str(test.loc[(test['position']==pos) & cond_pospts].reset_index(drop=True).iloc[countListPartial[pos]]['pts_proj']))
    print(f'{pos} Spot #{countList[pos]}: ' + str(test.loc[(test['position']==pos) & cond_pospts].reset_index(drop=True).iloc[countList[pos]]['pts_proj']))


QB Mean: 131.82
QB Median: 86.35
QB Spot #15: 237.18
QB Spot #24: 215.74
RB Mean: 113.78
RB Median: 105.29
RB Spot #35: 156.11
RB Spot #60: 88.05000000000001
WR Mean: 93.23
WR Median: 93.08
WR Spot #47: 135.57500000000002
WR Spot #72: 106.32900000000001
TE Mean: 51.99
TE Median: 40.3
TE Spot #35: 57.875
TE Spot #60: 30.585


In [23]:
for pos in ['QB', 'RB', 'WR', 'TE', 'PK', 'DEF']:
    

QB Median: 5.55
RB Median: 6.68
WR Median: 5.7
TE Median: 2.46
PK Median: 9.3
DEF Median: 4.87


# Find trade targets

In [35]:
chop_df = full.copy()
# Type player's name to find their vitals
chop = 'David Montgomery'
# See how bold the user wants to be
multiplier_tradeup = 1.05
# Calc across season
# Get relative points
chop_df = getRelProjs_optimized(chop_df)
# Sum points across the season
#chop_df = getSeasonTotals(chop_df)
chop_df


,player_id,full_name,birth_date,age,weight,height,team,position,fantasy_positions,depth_chart_order,...,date_prediction,pts_proj,owner_id,taxi,starters,players,user_id,franchise_name,display_name,rel_proj
0,4984,Josh Allen,1996-05-21,27.0,237.0,77,BUF,QB,['QB'],1.0,...,20230909,25.9000,904399374891855872,NaN,True,True,904399374891855872,Comeback 👑’s,comebackking15,26.078
1,4984,Josh Allen,1996-05-21,27.0,237.0,77,BUF,QB,['QB'],1.0,...,20230909,25.9000,904399374891855872,NaN,True,True,904399374891855872,Comeback 👑’s,comebackking15,25.900
2,4046,Patrick Mahomes,1995-09-17,27.0,225.0,74,KC,QB,['QB'],1.0,...,20230909,25.8600,904398822170636288,NaN,True,True,904398822170636288,Saskatoon Squatches,SpacemnSpiff,26.036
3,4046,Patrick Mahomes,1995-09-17,27.0,225.0,74,KC,QB,['QB'],1.0,...,20230909,25.8600,904398822170636288,NaN,True,True,904398822170636288,Saskatoon Squatches,SpacemnSpiff,25.860
4,4046,Patrick Mahomes,1995-09-17,27.0,225.0,74,KC,QB,['QB'],1.0,...,20230909,25.8500,904398822170636288,NaN,True,True,904398822170636288,Saskatoon Squatches,SpacemnSpiff,26.026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8340,MIN,MIN,NaN,NaN,NaN,NaN,MIN,DEF,['DEF'],NaN,...,20230909,-4.3633,904399374891855872,NaN,True,True,904399374891855872,Comeback 👑’s,comebackking15,0.000
8341,MIN,MIN,NaN,NaN,NaN,NaN,MIN,DEF,['DEF'],NaN,...,20230909,-5.6689,904399374891855872,NaN,True,True,904399374891855872,Comeback 👑’s,comebackking15,0.000
8342,CHI,CHI,NaN,NaN,NaN,NaN,CHI,DEF,['DEF'],NaN,...,20230909,-6.3891,452577139540094976,NaN,True,True,452577139540094976,2014champ,2014champ,0.000
8343,CHI,CHI,NaN,NaN,NaN,NaN,CHI,DEF,['DEF'],NaN,...,20230909,-7.5229,452577139540094976,NaN,True,True,452577139540094976,2014champ,2014champ,0.000


In [ ]:
# Find the player
chop_row = chop_df.loc[chop_df['full_name']==chop].head(1).reset_index().iloc[0]
#chop_id = chop_row['player_id']
chop_pos = chop_row['position']
chop_age = chop_row['age']
chop_rel = chop_row['rel_proj']
# Select
#cond_id = chop_df['player_id']!=chop_id
cond_rel_min = chop_df['rel_proj']>=chop_rel
cond_rel_max = chop_df['rel_proj']<=chop_rel * multiplier_tradeup
cond_notfa = chop_df['franchise_name'].notna()
targs = chop_df.loc[cond_rel_min & cond_rel_max & cond_notfa]
targs

# Write to CSV

In [98]:
# sums.to_csv(f'processed_{date_string}.csv', index=False)
waivers.to_csv(f'waivers_{date_string}.csv', index=False)

In [97]:
# Starter report
full.loc[full['starters']==True].groupby('franchise_name')['rel_pts'].sum().sort_values()

KeyError: 'Column not found: rel_pts'

In [96]:
full.loc[full['starters']==True].groupby('franchise_name')['pts_proj'].sum().sort_values()

franchise_name
DangeRUSS Last Ride          160.0080
Comeback 👑’s                1885.1302
kevinbash                   1925.8060
Idk much about soccer       2105.1885
Count of Monte Christian    2163.8135
2014champ                   2194.1737
GusTheBus                   2283.6228
📜 Providence 🪬 Spirits 🥂    2324.0345
Verdanks Vacqueros FFC      2426.5528
Pretty Big Wieners          2483.3059
Saskatoon Squatches         2582.2908
Croccity Body Snatchers     2733.0872
Name: pts_proj, dtype: float64